# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os

In [3]:
import zipfile

zip_file_path = 'Data/belt-exam-aml-mock-exam.zip'
extract_folder = 'Data/'

# Create the 'Data' folder if it dosen't exist
os.makedirs(extract_folder,exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)